# Optimizing the bucketing process

In [1]:
import pandas as pd

from skorecard.datasets import load_credit_card

df = load_credit_card(as_frame=True)

# Show
display(df.head(4))

num_feats = ['x1','x15','x16']

X = df[num_feats]
y = df['y']

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x15,x16,x17,x18,x19,x20,x21,x22,x23,y
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0


# Finding the best bucketing

The art of building a good scorecard model lies in finding the best bucketing strategy.<br>
Good buckets improve the predicitve power of the model, as well as guarantee stability of the predictions.<br>

This is normally a very manual, labour intensive process (and for a good reason).<br>

A good buckets follow the following principles:
- maximize the Information Value, defined as 

$$IV = \sum_{i}(\%G_{i}-\%B_{i})\dot\log(\frac{\%G_{i}}{\%B_{i}})$$

- avoid buckets that contain a very large or very small fraction of the population
- wherever the business sense requires it, 

The `skorecard` package provides some tooling to automate part of the process, namely:

- Grid search the hyper-parameters of the bucketers in order to maximise the information value
- Run the optimal bucketer within the bucketing process




## Grid search the bucketers to maximise the information value

`skorecard` implements an `IV_scorer`, that can be used as a custom scoring function for grid searching.<br>
The following snippets of code show how to integrate it in the grid search.<br>
The DecisionTreeBucketer applied on numerical features is the best use case, as there are some hyper-parameters that influence the bucketing quality. 

In [2]:
from skorecard.metrics import IV_scorer
from skorecard.bucketers import DecisionTreeBucketer
from sklearn.model_selection import GridSearchCV

The DecisionTreeBucketer has two main hyperparameters to grid-search:
- `max_n_bins`, maximum number of bins allowed for the bucketing
- `min_bin_size` minimum fraction of data in the buckets

In [3]:
gs_params = {
   "max_n_bins": [3, 4, 5, 6],
   "min_bin_size": [0.05, 0.06, 0.07, 0.08], #, 0.12]
}

The optimization has to be done for every feature indipendently, therefore we need a loop, and all the parameters are best stored in a data collector, like a dictionary

In [4]:
# Define the specials
best_params = dict()
max_iv = dict()
cv_results = dict()

# Add a special for demo purposes
specials = {'x1':{'special 0':['50000.0']}}

for feat in num_feats:

    #This snippet illustrates what to do with special values
    if feat in specials.keys():
        # This construct is needed to remap the specials, because skorecard validates that the key 
        # of the dictionary is present in the variables
        special = {feat: specials[feat]}
    else:
        special = {}
    bucketer = DecisionTreeBucketer(variables=[feat], specials=special)
    gs = GridSearchCV(bucketer, gs_params, scoring=IV_scorer, cv=3, return_train_score=True)
    gs.fit(X[[feat]], y)

    best_params[feat] = gs.best_params_
    max_iv[feat] = gs.best_score_
    cv_results[feat] = gs.cv_results_

Checking the best parameters per feature

In [5]:
best_params

{'x1': {'max_n_bins': 6, 'min_bin_size': 0.05},
 'x15': {'max_n_bins': 6, 'min_bin_size': 0.08},
 'x16': {'max_n_bins': 6, 'min_bin_size': 0.05}}

Because of its additive nature, IV is likely to be maximal for the highest `max_n_bins`. 
Therefore it is worth looking analysing the CV results!

In [6]:
cv_results['x1']

{'mean_fit_time': array([0.09358168, 0.08749469, 0.08900897, 0.08818277, 0.08763576,
        0.08831199, 0.08456341, 0.08533502, 0.08416621, 0.09112167,
        0.0896643 , 0.0931441 , 0.09968567, 0.09904973, 0.10114026,
        0.09465035]),
 'std_fit_time': array([0.00568708, 0.00219181, 0.00041746, 0.00208567, 0.00033371,
        0.00090009, 0.00135716, 0.00064736, 0.0008165 , 0.00254417,
        0.00449863, 0.00616635, 0.00619145, 0.0090274 , 0.00619416,
        0.00552724]),
 'mean_score_time': array([0.01997574, 0.02116672, 0.02150178, 0.02093689, 0.02005355,
        0.02037962, 0.0198973 , 0.02062575, 0.0202628 , 0.02284137,
        0.02007596, 0.02183072, 0.02206667, 0.02308766, 0.02521006,
        0.02146808]),
 'std_score_time': array([9.91640421e-05, 1.01564924e-03, 6.02460712e-04, 7.13789293e-04,
        4.51718100e-04, 1.02443455e-03, 1.56720318e-04, 1.20877918e-03,
        5.39789960e-04, 2.44794052e-03, 1.77551404e-04, 1.70836073e-03,
        1.53440960e-03, 1.20257386e-

## RandomizedSearchCV to maximise AUC
As `Skorecard` is scikit-learn compatibile we can use scikit-learn methods such as RandomizedSearchCV to maximise the AUC of our model. Shown below is one such example

In [7]:
from skorecard.bucketers import DecisionTreeBucketer, OptimalBucketer
from skorecard.pipeline import BucketingProcess
from skorecard.linear_model import LogisticRegression
from skorecard.preprocessing import WoeEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.pipeline import make_pipeline
from scipy.stats import uniform


def get_pipeline():
    
    bucketing_process = BucketingProcess(
        prebucketing_pipeline=make_pipeline(
                DecisionTreeBucketer(max_n_bins=100, min_bin_size=0.05),
        ),
        bucketing_pipeline=make_pipeline(
                OptimalBucketer(max_n_bins=10, min_bin_size=0.04),
        ),
    )

    return make_pipeline(
        bucketing_process,
        WoeEncoder(),
        LogisticRegression(solver="liblinear", C=1.7, max_iter=150, random_state=0) 
    )

pipe = get_pipeline()


param_grid = [
    {
        'logisticregression__C' : uniform(loc=0, scale=4),
        'logisticregression__solver' : ['liblinear']
    },
]

search_cv = RandomizedSearchCV(pipe, param_distributions = param_grid, cv = 5, verbose=True, scoring='roc_auc', n_jobs=-1, random_state=0, refit=True)
search_cv.fit(X, y)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('bucketingprocess',
                                              BucketingProcess(bucketing_pipeline=Pipeline(steps=[('optimalbucketer',
                                                                                                   OptimalBucketer(min_bin_size=0.04))]),
                                                               prebucketing_pipeline=Pipeline(steps=[('decisiontreebucketer',
                                                                                                      DecisionTreeBucketer())]))),
                                             ('woeencoder', WoeEncoder()),
                                             ('logisticregression',
                                              LogisticRegression(C=1.7,
                                                                 max_iter=150,
                                                                 random_state=0,
                     

In [8]:
search_cv.best_params_, search_cv.best_score_

({'logisticregression__C': 1.5337660753031108,
  'logisticregression__solver': 'liblinear'},
 0.6108804186848948)